In [15]:
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

### Read the data

In [2]:
df = pd.read_csv("sphist.csv")
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [3]:
df["Date"] = pd.to_datetime(df["Date"])
df.sort_values("Date", ascending=True, inplace=True)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


### Create **day_5** indicators
The average price from the past *5* days.

In [7]:
df["day_5"] = df["Close"].rolling(5).mean().shift(1)
print(df[df["Date"] == datetime(year=1951, month=1, day=16)])

            Date       Open       High        Low      Close     Volume  \
16330 1951-01-16  21.459999  21.459999  21.459999  21.459999  3740000.0   

       Adj Close   day_5  
16330  21.459999  21.114  


### Create day_30 indicators
The average price for the past *30* days.

In [8]:
df["day_30"] = df["Close"].rolling(30).mean().shift(1)
print(df[df["Date"] == datetime(year=1951, month=1, day=16)])

            Date       Open       High        Low      Close     Volume  \
16330 1951-01-16  21.459999  21.459999  21.459999  21.459999  3740000.0   

       Adj Close   day_5  day_30  
16330  21.459999  21.114  20.156  


### Create std_5 indicators
The standard deviation of the price over the past 5 days

In [9]:
df["std_5"] = df["Close"].rolling(5).std().shift(1)
print(df[df["Date"] == datetime(year=1951, month=1, day=16)])

            Date       Open       High        Low      Close     Volume  \
16330 1951-01-16  21.459999  21.459999  21.459999  21.459999  3740000.0   

       Adj Close   day_5  day_30     std_5  
16330  21.459999  21.114  20.156  0.165922  


Since you're computing indicators that use historical data, there are some rows where there isn't enough historical data to generate them. Some of the indicators use 365 days of historical data and the dataset starts on 1950-01-03. Thus, any rows that fall before 1951-01-03 don't have enough historical data to compute all the indicators. You'll need to remove these rows before you split the data.

#### Remove any rows from the DataFrame that occur before 1951-01-03

In [11]:
df_after_19510103 = df[df["Date"] >= datetime(year=1951, month=1, day=3)]
df_after_19510103.head()

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,day_30,std_5
16339,1951-01-03,20.690001,20.690001,20.690001,20.690001,3370000.0,20.690001,20.360000,19.815000,0.304385
16338,1951-01-04,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,20.514000,19.842666,0.204524
16337,1951-01-05,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,20.628000,19.874000,0.214057
16336,1951-01-08,21.000000,21.000000,21.000000,21.000000,2780000.0,21.000000,20.726001,19.907000,0.181879
16335,1951-01-09,21.120001,21.120001,21.120001,21.120001,3800000.0,21.120001,20.840001,19.935000,0.117047


In [12]:
df_after_19510103 = df_after_19510103.dropna(axis=0)

#### Generate train and test dataframe
Train should contain any rows in the data with a date less than 2013-01-01. test should contain any rows with a date greater than or equal to 2013-01-01.

In [13]:
train = df_after_19510103[df_after_19510103["Date"] < datetime(year=2013, month=1, day=1)]
test = df_after_19510103[df_after_19510103["Date"] >= datetime(year=2013, month=1, day=1)]

In [14]:
test.head()

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,day_30,std_5
738,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4.202600e+09,1462.420044,1418.641992,1414.258667,9.820801
737,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3.829730e+09,1459.369995,1425.793994,1417.676668,22.261321
736,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3.424290e+09,1466.469971,1433.702002,1420.092668,26.274326
735,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3.304970e+09,1461.890015,1443.376001,1422.714665,27.945242
734,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3.601600e+09,1457.150024,1455.267993,1425.076664,16.453319


### Making prediction

In [16]:
model = LinearRegression()
features = ["day_5", "day_30", "std_5"]
target = "Close"

model.fit(train[features], train[target])
prediction = model.predict(test[features])

mse = mean_absolute_error(test[target], prediction)
print(mse)

16.12624856941638


In [18]:
test['predict_close'] = prediction.copy()
test.head()

/Users/apple/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Date,Open,High,Low,Close,Volume,Adj Close,day_5,day_30,std_5,predict_close
738,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4.202600e+09,1462.420044,1418.641992,1414.258667,9.820801,1419.410758
737,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3.829730e+09,1459.369995,1425.793994,1417.676668,22.261321,1425.989848
736,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3.424290e+09,1466.469971,1433.702002,1420.092668,26.274326,1434.021243
735,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3.304970e+09,1461.890015,1443.376001,1422.714665,27.945242,1444.089259
734,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3.601600e+09,1457.150024,1455.267993,1425.076664,16.453319,1457.444496


We can improve the error of this model significantly, though. Think about some indicators that might be helpful to compute.

* The average volume over the past five days.
* The average volume over the past year.
* The ratio between the average volume for the past five days, and the average volume for the past year.
* The standard deviation of the average volume over the past five days.
* The standard deviation of the average volume over the past year.
* The ratio between the standard deviation of the average volume for the past five days, and the standard deviation of the average volume for the past year.
* The year component of the date.
* The ratio between the lowest price in the past year and the current price.
* The ratio between the highest price in the past year and the current price.
* The month component of the table.
* The day of week
* The day component of the table.
* The number of holidays in the prior month.

In [25]:
df["volume_5"] = df["Volume"].rolling(5).mean().shift(1)
df["std_volume_5"] = df["Volume"].rolling(5).std().shift(1)

df_after_19510103 = df[df["Date"] >= datetime(year=1951, month=1, day=3)]
df_after_19510103 = df_after_19510103.dropna(axis=0)

df_after_19510103.head()

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,day_30,std_5,volume_5,std_volume_5
16339,1951-01-03,20.690001,20.690001,20.690001,20.690001,3370000.0,20.690001,20.360000,19.815000,0.304385,3126000.0,370108.092319
16338,1951-01-04,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,20.514000,19.842666,0.204524,3268000.0,269016.728104
16337,1951-01-05,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,20.628000,19.874000,0.214057,3358000.0,197661.326516
16336,1951-01-08,21.000000,21.000000,21.000000,21.000000,2780000.0,21.000000,20.726001,19.907000,0.181879,3324000.0,166373.074745
16335,1951-01-09,21.120001,21.120001,21.120001,21.120001,3800000.0,21.120001,20.840001,19.935000,0.117047,3192000.0,276622.486432


In [26]:
model = LinearRegression()

train = df_after_19510103[df_after_19510103["Date"] < datetime(year=2013, month=1, day=1)]
test = df_after_19510103[df_after_19510103["Date"] >= datetime(year=2013, month=1, day=1)]

features = ["day_5", "day_30", "std_5", "volume_5"]
target = "Close"

model.fit(train[features], train[target])
prediction = model.predict(test[features])

mse = mean_absolute_error(test[target], prediction)
print(mse)

16.126509218367058


There's a lot of improvement to be made on the indicator side and we urge you to think of better indicators that you could use for prediction. We can also make significant structural improvements to the algorithm and pull in data from other sources.

* Accuracy would improve greatly by making predictions only one day ahead. For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what you'd do if you were trading using the algorithm.
* You can also improve the algorithm used significantly. Try other techniques, like a random forest, and see if they perform better.
* You can also incorporate outside data, such as the weather in New York City (where most trading happens) the day before and the amount of Twitter activity around certain stocks.
* You can also make the system real-time by writing an automated script to download the latest data when the market closes and make predictions for the next day.
* Finally, you can make the system "higher-resolution". You're currently making daily predictions, but you could make hourly, minute-by-minute, or second-by-second predictions. This requires obtaining more data, though. You could also make predictions for individual stocks instead of the S&P500.